In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar

In [2]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

# Import 2021 Picks

In [27]:
df = pd.read_csv('backend\\data\\predictions\\2021_nn.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2021, 9, 1)]
df['hit'] = df['prediction'] == df['outcome']
df.dropna(inplace=True)

## Betting Analysis 

In [32]:
# Per Month
df['month'] = df['date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
per_month = df.groupby(['month']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_month['perc'] = (per_month[('hit', 'sum')] / per_month[('hit', 'count')] * 100).apply(round)
per_month.sort_values(by=[('profit', 'sum'), ], ascending=False)

profit hit       perc
             sum sum count     
month                          
Oct    21.126983  47    72   65
Nov    16.051695  32    59   54
Dec    14.755933  37    60   62
Sep     7.155428  30    49   61
Feb     1.315789   1     1  100
Jan   -31.356804  20    44   45

In [33]:
# Per Week
per_week = df.groupby(['week']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_week['perc'] = (per_week[('hit', 'sum')] / per_week[('hit', 'count')] * 100).apply(round)
per_week

profit hit       perc
            sum sum count     
week                          
1     -3.849153   8    16   50
2     -2.111897  10    16   62
3     11.033145  11    16   69
4     -5.190927   8    16   50
5     14.398209  14    16   88
6      3.470359   9    14   64
7     -5.273689   7    13   54
8     16.092078  11    15   73
9    -10.256638   5    14   36
10    29.108547  10    14   71
11    -1.342374   8    15   53
12    -1.743556   8    15   53
13    -6.511265   7    14   50
14     7.347792  12    14   86
15     5.305403  10    16   62
16     8.614003   8    16   50
17    -0.859819  11    16   69
18    -9.494689   7    16   44
19    -8.002296   2     6   33
20    -9.500000   0     4    0
21    -3.500000   0     2    0
22     1.315789   1     1  100

In [34]:
# Per Team
per_team = df.groupby(['prediction']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_team['perc'] = (per_team[('hit', 'sum')] / per_team[('hit', 'count')] * 100).apply(round)
per_team.sort_values(by=[('profit', 'sum'), ], ascending=False)

profit hit       perc
                                sum sum count     
prediction                                        
Pittsburgh Steelers       10.548349   6     9   67
Arizona Cardinals          9.297814   9    12   75
Tennessee Titans           9.211957   6     9   67
Houston Texans             9.125000   2     3   67
Chicago Bears              8.885714   4     6   67
Green Bay Packers          8.196337  11    15   73
Los Angeles Rams           7.640903  12    14   86
Tampa Bay Buccaneers       7.533393  12    14   86
San Francisco 49ers        5.928445   9    13   69
Los Angeles Chargers       5.927832   6    10   60
Washington Football Team   5.701852   5     8   62
Cleveland Browns           5.440789   3     4   75
Kansas City Chiefs         5.069439  13    15   87
Buffalo Bills              2.337983   7    10   70
Baltimore Ravens           1.698430   5    10   50
Dallas Cowboys             1.401620   6     9   67
New England Patriots       1.150818   9    14   64
New York Giants           -0.250000   0     1    0
Philadelphia Eagles       -0.985539   6    10   60
Minnesota Vikings         -1.318411   5     9   56
Jacksonville Jaguars      -1.500000   0     1    0
Cincinnati Bengals        -1.725877   3     8   38
New York Jets             -1.750000   2     6   33
New Orleans Saints        -4.125988   4    10   40
Atlanta Falcons           -5.313183   3     6   50
Carolina Panthers         -6.350000   3     9   33
Las Vegas Raiders         -6.465517   1     4   25
Miami Dolphins            -7.631883   4    10   40
Seattle Seahawks          -7.647059   3     9   33
Indianapolis Colts        -9.018939   3    10   30
Denver Broncos            -9.965256   4     9   44
Detroit Lions            -12.000000   1     8   12